In [1]:
#coding=utf-8

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

#compile sym_talk dictionary
talk_dic ={}
f_talk = open('symptom_talks.txt', encoding = 'utf-8')
for talk_line in f_talk:  
    talk_words = talk_line.replace('\n', '')
    split_talk = talk_words.split(':')
    talk_dic[split_talk[0]]=split_talk[1]

#compile doc-word set
f = open('./new_SymptomToOutpatient.txt', encoding = 'utf-8').read()
splitF = f.split(':')
words = [w.replace('\n', '') for w in splitF]
depgroup =set()
for dep in words:
    depname = dep.split(',')
    depgroup.update(depname[:-1])
#print(depgroup)

new_f = open('new_SymptomToOutpatient.txt', encoding = 'utf-8')
doc_list= []
for deps in depgroup: 
    slist=[]
    new_f.seek(0)
    for line in new_f:
        sym_dep = line.split(":")
        depset = sym_dep[1].split(",")
        for dep in depset:
            if(dep==deps):
                slist.append(sym_dep[0])
    joined = " ".join(slist)
    doc_list.append(joined)

wf = open('word_doc.txt','w', encoding = 'utf-8')
for dw in doc_list:
    wf.write(dw + "\n")

wf_new = open('word_doc.txt', encoding = 'utf-8')  
doccontent=[]
for line in wf_new:
    line = line.replace("\n","")
    doccontent.append(line)
    
vectorizer = TfidfVectorizer()                  
tfidf = vectorizer.fit_transform(doccontent).toarray()            
vocab = vectorizer.get_feature_names()

#age question
age_answer = input('How old are you? ')
if int(age_answer) < 18:
    print("衛生署宣導：十八歲以下的人士，仍建議看兒科，台灣兒科醫學會也表示，18 歲前的兒童與青少年，身心皆處於發育階段，發病症狀與治療方式與成人有所差異，請選擇至兒科院所就診")
        
sex_answer = input('are you male or female (male= 1 or female=2) ')
if sex_answer == "1":
    sex = "Male"
    print("先生您好!")
if sex_answer == "2":
    sex = "Female" 
    print("小姐您好!")
    
test_set=[]
print("症狀:" + str(vocab))
initial_sym = input('please write the initial symptons (splited by space):')
print("您的初始症狀是:" + initial_sym)
test_set.append(initial_sym)
query_set = []
split_sym=test_set[0].split(" ")
for symid in range(len(split_sym)):
    query_set.append(split_sym[symid])

if sex=="Female":
    query_set.append("睾丸疼痛")
    query_set.append("遺精")
else:
    query_set.append("乳房腫塊")
    query_set.append("白帶")
    query_set.append("閉經")
    query_set.append("痛經")
    query_set.append("宮頸糜爛")
    query_set.append("流產")

testVectorizerArray = vectorizer.transform(test_set).toarray()
#初始症狀與科別的cosine_similarity
cossim = cosine_similarity(testVectorizerArray, tfidf)
print(cossim)

#nonzero_list = np.nonzero(cossim)
nonzero_list = []
#Add cosine similarity score restriction before np.nonzero
score = 0.2  #change score  
for simid in range(len(cossim[0])):
    if float(cossim[0][simid])>score:
        nonzero_list.append(simid)
#print(nonzero_list)
argsort = np.argsort(cossim)
#print(argsort)

a = [argsort[0][i] for i in range(len(argsort[0])) if argsort[0][i] in nonzero_list]
for i in a:
    print(list(depgroup)[i])

newlist = []
for depid in a:
    newlist.append(doccontent[depid])

tfvectorizer = TfidfVectorizer()                       
X = tfvectorizer.fit_transform(newlist)
features = tfvectorizer.get_feature_names()

def top_tfidf_feats(row, features, top_n=10):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [features[i] for i in topn_ids]
    df = [(features[i], row[i]) for i in topn_ids]
    feature_df = pd.DataFrame(df)
    feature_df.columns = ['feature', 'tfidf']
    print(feature_df)
    return top_feats

def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=10):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.max(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)

top_feats = top_mean_feats(X,features, top_n=30)
choice = False
answer = "-1"
if len(a)==1: 
    choice=True
    print("recommand:" + str(list(depgroup)[a[len(a)-1]]) )
final_score=0.7
totalask = 0
while(float(cossim[0][a[len(a)-1]])<final_score and totalask < 10 and choice==False):       #條件cosine similarity<預設score and 問題次數為否
    totalask +=1
    #print("query_set" + str(query_set))
    if answer=="1":
        print("answer 1" + test_set[0])
        test_set[0] = test_set[0] + " " + query
        print("test_set" + str(test_set))
        # rerun newlist
        testVectorizerArray = vectorizer.transform(test_set).toarray()
        cossim = cosine_similarity(testVectorizerArray, tfidf)
        print(cossim)
        #nonzero_list = np.nonzero(cossim)
        nonzero_list = []
        for simid in range(len(cossim[0])):
            if float(cossim[0][simid])>score:
                nonzero_list.append(simid)
        argsort = np.argsort(cossim)
        print(nonzero_list)
        a = [argsort[0][i] for i in range(len(argsort[0])) if argsort[0][i] in nonzero_list]
        for i in a:
            print(list(depgroup)[i]) 
    
        newlist = []    
        for depid in a:
            newlist.append(doccontent[depid])
        #print(newlist)
        tfvectorizer = TfidfVectorizer()                       
        X = tfvectorizer.fit_transform(newlist)
        features = tfvectorizer.get_feature_names()
        #print('Transform Vectorizer to test set' , str(X.shape))
        top_feats = top_mean_feats(X,features, top_n=10)
        query = ""
        for i in range(len(top_feats)):
            if(top_feats[i] not in query_set):
                query = top_feats[i]
                query_talk = talk_dic[query]
                query_set.append(top_feats[i])
                break
     
    # check top deps > definded_count --> choice to false
    elif answer=="-1":
        #print("answer -1")
        query = ""
        # check top deps > definded_count --> choice to false
        for i in range(len(top_feats)):
            if(top_feats[i] not in query_set):
                query = top_feats[i]
                query_talk = talk_dic[query]
                query_set.append(top_feats[i])
                break
    else:
        print("please type (1 or -1)")
        totalask -=1
    answer = input(query_talk+'(1 or -1) ')
    
if int(age_answer)>18 and str(list(depgroup)[a[len(a)-1]])=="小兒科":
    print("recommand:家醫科" )
else:
    print("recommand:" + str(list(depgroup)[a[len(a)-1]]) ) 


How old are you? 19
are you male or female (male= 1 or female=2) 2
小姐您好!
症狀:['乏力', '乳頭內陷', '休克', '低血壓', '便秘', '便血', '全身酸痛', '創傷', '反胃', '口臭', '吞咽困難', '呼吸困難', '咯血', '咳痰', '咳血', '嘔吐', '嘔血', '噁心', '宮頸糜爛', '少尿', '尿失禁', '尿痛', '尿血', '尿頻', '心絞痛', '性交疼痛', '惡寒', '抽搐', '抽筋', '氣喘', '流產', '消化不良', '濕疹', '無尿', '玫瑰疹', '畏光', '疼痛', '痢疾', '發燒', '白帶', '皰疹', '盜汗', '睾丸疼痛', '羅音', '羊水過多', '耳痛', '耳鳴', '肝掌', '胃痛', '胸悶', '腹痛', '腹脹', '血壓升高', '貧血', '遺尿', '遺精', '閉經', '關節痛', '雀斑', '雞眼', '靜脈曲張', '食慾減退', '食欲不振', '鱗屑', '黃疸', '鼻出血', '鼻塞']
please write the initial symptons (splited by space):發燒 便秘
您的初始症狀是:發燒 便秘
[[ 0.46198966  0.28456037  0.27323693  0.24110756  0.32783848  0.25354606
   0.          0.23347064  0.2413552   0.21137725  0.29558804  0.29642646
   0.10380525  0.26232335  0.17764008  0.24623923  0.2249162   0.36413373]]
內分泌科
腎內科
泌尿外科
心血管內科
風濕科
傳染科
心胸外科
急診科
眼科
耳鼻喉科
呼吸內科
肝膽外科
手外科
消化內科
小兒科
   feature     tfidf
0       創傷  0.724697
1       黃疸  0.681392
2       畏光  0.637671
3     呼吸困難  0.532559
4       耳鳴  0.53027